# 로지스틱 회귀 Logistic Regression

- 선형 회귀와 동일하게 선형 방정식을 학습
- 분류 문제를 해결하기 위해 사용하는 통계적 방법
- 결과 변수가 0 또는 1인 경우에 사용

### 시그모이드 함수 Sigmoid Function 또는 로지스틱 함수 Logistic Fuction
- 종속 변수를 0과 1 사이의 확률로 변환하기 위해 사용
- 종속 변수가 아주 큰 음수일 때 0, 종속 변수가 아주 큰 양수일 때 1이 되도록 함
- 이를 통해 선형 결합의 값을 확률로 해석

In [1]:
import pandas as pd
import numpy as np

### decision funvction()
- 로지스틱 회귀에서 각 샘플에 대해 decision boundary로부터의 거리를 반환
- 각 샘플이 양성 클래서에 속할지 음성 클래스에 속할지 결정
- decision function의 결과는 위 가중치와 편향을 반영한 방정식의 결과를 종속변수로 반환
- 방정식 결과의 종속 변수가 클수록 양성 클래스일 가능성이 높아지고, 작을수록 음성 클래스에 속할 가능성이 높아짐

In [3]:
fish = pd.read_csv(r'https://bit.ly/fish_csv')
fish

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [8]:
# 데이터 저장
fish_target = fish['Species'].to_numpy()
print(fish_target[:3])

fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:3])

['Bream' 'Bream' 'Bream']
[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]]


In [9]:
# 훈련 데이터, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

### Stadard Scaler 하는 이유
- feature들의 단위가 다르기 때문에 동일한 스케일로 맞춰서 모델 성능 향상을 꿰함 

In [10]:
# 데이터 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [11]:
print(train_scaled[:3])
print(test_scaled[:3])

[[ 0.91965782  0.60943175  0.81041221  1.85194896  1.00075672]
 [ 0.30041219  1.54653445  1.45316551 -0.46981663  0.27291745]
 [-1.0858536  -1.68646987 -1.70848587 -1.70159849 -2.0044758 ]]
[[-0.88741352 -0.91804565 -1.03098914 -0.90464451 -0.80762518]
 [-1.06924656 -1.50842035 -1.54345461 -1.58849582 -1.93803151]
 [-0.54401367  0.35641402  0.30663259 -0.8135697  -0.65388895]]


In [13]:
# Logistic Regression을 이용해 이진 분류 수행
# 데이터 필터링
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
# bream, smelt 일 때만, Treu 그외는 False

In [14]:
# Logistic Regression 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [15]:
# 훈련된 모델을 이용한 예측된 클래스 반환
lr.predict(train_bream_smelt[:5])

array(['Bream', 'Smelt', 'Bream', 'Bream', 'Bream'], dtype=object)

In [17]:
# 학습한 클래스
lr.classes_

array(['Bream', 'Smelt'], dtype=object)

In [16]:
# 입력 데이터에 대해 각 클래스에 속할 확률 반환
lr.predict_proba(train_bream_smelt[:5])

# 첫 행 : bream에 속할 확률
# 두번째 행 : smelt에 속할 확률

array([[0.99760007, 0.00239993],
       [0.02737325, 0.97262675],
       [0.99486386, 0.00513614],
       [0.98585047, 0.01414953],
       [0.99767419, 0.00232581]])

### 가중치가 여러개
- 모델이 여러 특성을 기반으로 예측

In [19]:
print('가중치 :', lr.coef_)
print('편향 :', lr.intercept_)

가중치 : [[-0.40451732 -0.57582787 -0.66248158 -1.01329614 -0.73123131]]
편향 : [-2.16172774]


In [21]:
# decision function을 이용해 종속 변수 찾아보기
decisions = lr.decision_function(train_bream_smelt)
decisions

array([-6.02991358,  3.57043428, -5.26630496, -4.24382314, -6.06135688,
        3.41099831, -3.58967782, -7.66931655, -4.31727007, -2.22341548,
       -6.4698283 , -6.03601551, -5.77895847,  3.23601973, -7.81091675,
        3.36381609,  2.51455071, -3.88828092, -7.39580986, -3.51427836,
       -3.99094045, -4.17926154, -3.25374833, -5.0242727 , -4.23461336,
        3.18456426, -5.44264864,  3.34182307,  3.40394191, -2.76321242,
        2.48794425, -5.20693508, -4.77384768])

In [22]:
# signmoid F을 이용해 확률로 변환
from scipy.special import expit
expit(decisions)

array([2.39992835e-03, 9.72626754e-01, 5.13613552e-03, 1.41495320e-02,
       2.32581351e-03, 9.68046497e-01, 2.68655405e-02, 4.66718913e-04,
       1.31607262e-02, 9.76673891e-02, 1.54709455e-03, 2.38536358e-03,
       3.08240349e-03, 9.62167489e-01, 4.05122141e-04, 9.66554359e-01,
       9.25155604e-01, 2.00694898e-02, 6.13442824e-04, 2.89086870e-02,
       1.81469269e-02, 1.50789532e-02, 3.71924297e-02, 6.53340168e-03,
       1.42785787e-02, 9.60249253e-01, 4.30935384e-03, 9.65836049e-01,
       9.67827503e-01, 5.93447853e-02, 9.23292333e-01, 5.44858922e-03,
       8.37704538e-03])

- lr.predict_proba의 두번째 열의 결과 동일

### 로지스틱 회귀로 다중 분류 수행하기

In [23]:
# 모델 초기화
# max_iter에서 반복 횟수를 지정하여 모델 훈련
# 로지스틱 회귀의 규제를 완화하기 위해 숫자를 높임, C 로 설정
lr = LogisticRegression(C = 20, max_iter=1000)

# 모델 학습
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [24]:
lr.predict(test_scaled)

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch', 'Bream', 'Smelt',
       'Roach', 'Perch', 'Pike', 'Bream', 'Perch', 'Bream', 'Parkki',
       'Bream', 'Bream', 'Perch', 'Perch', 'Perch', 'Bream', 'Smelt',
       'Bream', 'Bream', 'Bream', 'Bream', 'Perch', 'Perch', 'Roach',
       'Smelt', 'Smelt', 'Pike', 'Perch', 'Perch', 'Pike', 'Bream',
       'Perch', 'Roach', 'Roach', 'Parkki', 'Perch'], dtype=object)

In [28]:
# 샘플에 대한 예측 확률
proba = lr.predict_proba(test_scaled)
np.round(proba, decimals = 3)

array([[0.   , 0.014, 0.842, 0.   , 0.135, 0.007, 0.003],
       [0.   , 0.003, 0.044, 0.   , 0.007, 0.946, 0.   ],
       [0.   , 0.   , 0.034, 0.934, 0.015, 0.016, 0.   ],
       [0.011, 0.034, 0.305, 0.006, 0.567, 0.   , 0.076],
       [0.   , 0.   , 0.904, 0.002, 0.089, 0.002, 0.001],
       [0.999, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.05 , 0.   , 0.004, 0.945, 0.   ],
       [0.001, 0.007, 0.319, 0.014, 0.639, 0.001, 0.018],
       [0.   , 0.002, 0.803, 0.002, 0.181, 0.   , 0.011],
       [0.   , 0.   , 0.022, 0.978, 0.   , 0.   , 0.   ],
       [0.984, 0.003, 0.   , 0.   , 0.001, 0.   , 0.013],
       [0.047, 0.   , 0.817, 0.008, 0.002, 0.   , 0.126],
       [0.992, 0.005, 0.   , 0.   , 0.   , 0.   , 0.003],
       [0.007, 0.907, 0.002, 0.   , 0.076, 0.   , 0.009],
       [0.999, 0.001, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.001, 0.   , 0.936, 0.   , 0.   , 0.   , 0.063],
       [0.001,

- 위의 결과는 모든 Species에 대한 예측 확률
- 한 행당 높은 확률에 해당하는 species(열)일 가능성이 높음

In [29]:
lr.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [31]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


- (7, 5)
    - 각 클래스에 대한 feature의 가중치
    - 7 : 클래스 개수
    - 5 : 각 클래스에 대해 5개의 특성feature를 가졌다는 의미
        - 입력 데이터가 5개의 특성을 가지고 있음
- (7, )
    - 각 클래스에 대한 절편
    - 7개의 클래스 각각에 대해 하나의 절편을 가짐
    - 7개의 절편 값을 가진 1차원 배열

In [32]:
# 로지스틱 회귀 모델이 각 클래스에 대해 결정 함수 값을 반환
# 
decision = lr.decision_function(test_scaled)
np.round(decision, decimals=2)

array([[-6.510e+00,  1.040e+00,  5.170e+00, -2.760e+00,  3.340e+00,
         3.500e-01, -6.300e-01],
       [-1.088e+01,  1.940e+00,  4.780e+00, -2.420e+00,  2.990e+00,
         7.840e+00, -4.250e+00],
       [-4.340e+00, -6.240e+00,  3.170e+00,  6.480e+00,  2.360e+00,
         2.430e+00, -3.870e+00],
       [-6.900e-01,  4.500e-01,  2.640e+00, -1.210e+00,  3.260e+00,
        -5.700e+00,  1.260e+00],
       [-6.400e+00, -1.990e+00,  5.820e+00, -1.300e-01,  3.500e+00,
        -9.000e-02, -7.000e-01],
       [ 1.669e+01,  8.310e+00, -3.350e+00, -4.950e+00,  8.500e-01,
        -2.634e+01,  8.790e+00],
       [-1.235e+01,  1.920e+00,  5.760e+00, -2.920e+00,  3.180e+00,
         8.690e+00, -4.280e+00],
       [-2.620e+00, -8.500e-01,  2.900e+00, -2.100e-01,  3.600e+00,
        -2.850e+00,  2.000e-02],
       [-3.840e+00, -1.060e+00,  4.990e+00, -9.600e-01,  3.500e+00,
        -3.370e+00,  7.400e-01],
       [ 1.670e+00, -1.450e+01,  1.047e+01,  1.424e+01, -9.300e-01,
        -1.205e+01,  1.

## softmax function
- 입력 값들을 확률 분포로 변화하여 각 클래스에 대한 확률로 반환
- 여러 개의 선형 방정식의 출력값을 0~1 사이로 압축하고 전체 합이 1이 되도록 함
- 지수 함수 사용
    - 모든 값을 양수로 만들고, 값 사이의 상대적인 차이 강조
- 정규화
    - 지수 함수 결과를 모두 더한 값으로 나눔
    - 출력 값이 1이 되도록 하기 위함

In [33]:
from scipy.special import softmax
# softmax의 axis를 이용해 소프트맥스를 계산할 축을 지정
# 앞수 구한 proba랑 결과 동일
proba = softmax(decision, axis = 1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]
 [0.999 0.    0.    0.    0.    0.    0.   ]
 [0.    0.001 0.05  0.    0.004 0.945 0.   ]
 [0.001 0.007 0.319 0.014 0.639 0.001 0.018]
 [0.    0.002 0.803 0.002 0.181 0.    0.011]
 [0.    0.    0.022 0.978 0.    0.    0.   ]
 [0.984 0.003 0.    0.    0.001 0.    0.013]
 [0.047 0.    0.817 0.008 0.002 0.    0.126]
 [0.992 0.005 0.    0.    0.    0.    0.003]
 [0.007 0.907 0.002 0.    0.076 0.    0.009]
 [0.999 0.001 0.    0.    0.    0.    0.   ]
 [1.    0.    0.    0.    0.    0.    0.   ]
 [0.001 0.    0.936 0.    0.    0.    0.063]
 [0.001 0.003 0.784 0.009 0.19  0.    0.014]
 [0.001 0.035 0.675 0.004 0.263 0.001 0.021]
 [0.991 0.005 0.    0.    0.    0.    0.003]
 [0.    0.002 0.049 0.    0.008 0.941 0.   ]
 [0.997 0.002 0.    0.    0.    0.    0.   ]
 [0.984 0.

### 결과
## 로지스틱 회귀로 확률 예측
- 예측과 예측의 근거가 되는 확률을 출력
- 회귀 모델이 아닌 분류 모델
- 선형 회귀처럼 선형 방정식 사용
- 계산한 값을 0~11 사이로 압축
    - 이 값은 0~100% 사이의 확률로 이해
# 이진 분류
- 하나의 선형 방정식 훈련
- 출력값을 시그모이드 함수에 통과시켜 0~1 사이값 만듬
    - 양성 클래스에 대한 확률
    - 음성 클래스 확률 = 1 - 양성 클래스 확률
# 다중 분류
- 클래스 개수만큼 방정식 훈련
- 출력값을 소프트맥스 함수를 통과시켜 전체 클래스에 대한 합이 항상 1이 되도록 함